## Install required python libraries

In [ ]:
!pip install requests

In [ ]:
!pip install psycopg2-binary

In [ ]:
!pip install pandas

## Import the libraries

In [1]:
import json
import psycopg2
import requests
import time
import logging
import pandas as pd

## Initialize database constants
> Enter the `database` & `username` created in CockroachDB.

> Enter the `tablename` you wish to create.

In [2]:
database = "employees"
username = "maxroach"
host = "localhost"
port = 26257
tablename = "jsontbl"

## Initialize API
> We will be using `http://dummy.restapiexample.com/api/v1/employees` for the demonstration purpose.

> Sample Json Output:

```json
{
    "status": "success",
    "data": [
        {
            "id": "1",
            "employee_name": "...",
            "employee_salary": "...",
            "employee_age": "...",
            "profile_image": ""
        },
        {
            "id": "2",
            "employee_name": "...",
            "employee_salary": "...",
            "employee_age": "...",
            "profile_image": ""
        },
        { 
            ... 
        }
    ]
}
```

In [3]:
url = "http://dummy.restapiexample.com/api/v1/employees"

## Create a connection to CockroachDB on Red Hat Marketplace

In [4]:
conn = psycopg2.connect(database=database, user=username, host=host, port=port)
conn.set_session(autocommit=True)
cur = conn.cursor()

## Create a table in CockroachDB

In [5]:
def create_table(conn):
    with conn.cursor() as cur:
        cur.execute('CREATE TABLE IF NOT EXISTS {} ( \
        id UUID DEFAULT uuid_v4()::UUID PRIMARY KEY, \
        posts JSONB \
        );'.format(tablename))
        logging.debug("create_table(): status message: {}".format(cur.statusmessage))
    conn.commit()
    print("Table {} created successfully!".format(tablename))

In [6]:
create_table(conn)

Table jsontbl created successfully!


## Make API call and store the response

In [7]:
req = requests.get(url, headers={"User-Agent": "Python"})

# Decode the JSON.
resp = req.json()

# Convert the JSON to a string to send to the database.
data = json.dumps(resp)
print(json.dumps(resp, indent=2))

{
  "status": "success",
  "data": [
    {
      "id": "1",
      "employee_name": "Tiger Nixon",
      "employee_salary": "320800",
      "employee_age": "61",
      "profile_image": ""
    },
    {
      "id": "2",
      "employee_name": "Garrett Winters",
      "employee_salary": "170750",
      "employee_age": "63",
      "profile_image": ""
    },
    {
      "id": "3",
      "employee_name": "Ashton Cox",
      "employee_salary": "86000",
      "employee_age": "66",
      "profile_image": ""
    },
    {
      "id": "4",
      "employee_name": "Cedric Kelly",
      "employee_salary": "433060",
      "employee_age": "22",
      "profile_image": ""
    },
    {
      "id": "5",
      "employee_name": "Airi Satou",
      "employee_salary": "162700",
      "employee_age": "33",
      "profile_image": ""
    },
    {
      "id": "6",
      "employee_name": "Brielle Williamson",
      "employee_salary": "372000",
      "employee_age": "61",
      "profile_image": ""
    },
    {
      

##### We structure our query so that we extract the `data` field, and then expand that and insert each individual element into the database as a separate row.

In [8]:
cur.execute("""INSERT INTO bank.jsontbl (posts)
        SELECT json_array_elements(%s->'data')""", (data,))

## Query the results from the table

In [9]:
def print_content(conn):
    with conn.cursor() as cur:
        cur.execute("SELECT * FROM {}".format(tablename))
        logging.debug("print_content(): status message: {}".format(cur.statusmessage))
        rows = cur.fetchall()
        return rows 

In [10]:
result = print_content(conn)

## Read the queried unstructured JSON in Pandas Dataframe

In [11]:
df = pd.DataFrame(columns = ['Employee_ID', 'Employee_Name', 'Employee_Salary', 'Employee_Age'])

In [12]:
idx = 0
for i in result:
    df.loc[len(df)] = [i[1]['id'], i[1]['employee_name'], i[1]['employee_salary'], i[1]['employee_age']]

In [13]:
df.head(10)

,Employee_ID,Employee_Name,Employee_Salary,Employee_Age
0,17,Paul Byrd,725000,64
1,6,Brielle Williamson,372000,61
2,9,Colleen Hurst,205500,39
3,19,Bradley Greer,132000,41
4,22,Yuri Berry,675000,40
5,4,Cedric Kelly,433060,22
6,22,Yuri Berry,675000,40
7,6,Brielle Williamson,372000,61
8,10,Sonya Frost,103600,23
9,5,Airi Satou,162700,33


## Close the connection to the CockroachDB Operator

In [14]:
cur.close()
conn.close()

## Summary

We learn't how to get unstructured data from a REST API, created a table in CockroachDB and stored the unstructured Json data into the table. We also learn't to query the unstructured data from CockroachDB table into a pandas dataframe.

## Future Scope

The pandas dataframe can be further used to clean, cleanse and refine the data. The data can be used to build Machine Learning models as well.

#### (Optional) Delete the records from the table in CockroachDB

In [ ]:
def delete_table_contents(conn):
    with conn.cursor() as cur:
        cur.execute("DELETE FROM {}".format(tablename))
        logging.debug("delete_table_contents(): status message: {}".format(cur.statusmessage))
    conn.commit()

In [ ]:
#delete_table_contents(conn)

#### (Optional) Delete the table from CockroachDB

In [ ]:
def drop_table(conn):
    with conn.cursor() as cur:
        cur.execute("DROP TABLE {}".format(tablename))
        logging.debug("drop_table(): status message: {}".format(cur.statusmessage))
    conn.commit()

In [ ]:
#drop_table(conn)